# **Householder Notebook**

Maby by eti.calde

---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

## **1.1 Householder Reflection**

**Definition**
Given a unit vector $u$ defining a hyperplane perpendicular to ir, a Householder reflection over a vector $x$ is defined as:

$$
H_{\mathbf{u}}(x) = x-2 \langle \mathbf{x},\mathbf{u}\rangle \mathbf{u}
$$

Is nothing more that the vector reflected over the hyperplane. This can be expressed in matrix form as follows:

$$
H =(I-2\mathbf{u}^T\mathbf{u})
$$

---

## **1.2 let’s see the transformation in code!!**

In [2]:
def get_householder_matrix(v):
    """
    Calculates the Householder reflection matrix for a vector v
    
    Parameters
    ----------
    v : ndarray (n,), input vector.
    Returns
    -------
    H : ndarray (n,n), Householder reflection matrix.
    """
    v = v.reshape(-1, 1)
    I = np.eye(v.shape[0])
    return I - 2 * (v @ v.T) / (v.T @ v)

In [3]:
cmap = plt.get_cmap("magma")
C_V1   = cmap(0.15)  # x
C_V2   = cmap(0.35)  # u
C_PLN  = cmap(0.85)  # hyperplane
C_REF  = cmap(0.60)  # H(x)

def plot_householder(theta1_deg=30.0, theta2_deg=120.0, r1=1.5):
    # Parameters
    t1 = np.deg2rad(theta1_deg)
    t2 = np.deg2rad(theta2_deg)
    unitari_rad = 1.0

    # Vectors
    x = np.array([r1*np.cos(t1), r1*np.sin(t1)])
    u = np.array([unitari_rad*np.cos(t2), unitari_rad*np.sin(t2)])

    # Hyperplane
    n_perp = np.array([-u[1], u[0]])

    # Householder
    H = get_householder_matrix(u)
    x_householder = H @ x
    print("x:", x)
    print("x' (Householder):", x_householder)
    # ===================================================================
    # PLOT
    plt.figure()

    # Vectors
    plt.quiver(0, 0, x[0], x[1], color=C_V1, scale=1, scale_units='xy', angles='xy', label='x')
    plt.quiver(0, 0, u[0], u[1], color=C_V2, scale=1, scale_units='xy', angles='xy', label='u (unitario)')
    plt.quiver(0, 0, x_householder[0], x_householder[1],
               color=C_REF, scale=1, scale_units='xy', angles='xy', label='H(x)')
    # Hyperplane
    L = 2.2
    pts = np.vstack([-L*n_perp, L*n_perp])
    plt.plot(pts[:,0], pts[:,1], linestyle='--', color=C_PLN, label='Hip. u·x = 0')

    # Details
    lim = 2.2
    plt.xlim(-lim, lim); plt.ylim(-lim, lim)
    plt.grid(True)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.title("Householder reflection of x")
    plt.show()

angle1 = FloatSlider(value=30.0,  min=0.0, max=360.0, step=1.0, description='Ángulo x (°)')
angle2 = FloatSlider(value=120.0, min=0.0, max=360.0, step=1.0, description='Ángulo u (°)')
len_r1 = FloatSlider(value=1.5,   min=0.0, max=2.0,   step=0.1, description='||x||')

interact(plot_householder, theta1_deg=angle1, theta2_deg=angle2, r1=len_r1)


interactive(children=(FloatSlider(value=30.0, description='Ángulo x (°)', max=360.0, step=1.0), FloatSlider(va…

<function __main__.plot_householder(theta1_deg=30.0, theta2_deg=120.0, r1=1.5)>

Now let's apply the Householder Reflection to get the QR factorization of a matrix $A$.

---

## **2.2 Get a random matrix**

In [4]:
def _random_orthogonal(n, rng):
    """Matriz ortogonal ~ Haar usando QR sobre una matriz gaussiana."""
    M = rng.standard_normal((n, n))
    Q, R = np.linalg.qr(M)
    # Asegurar unicidad de signos (evita que R tenga diag negativa)
    d = np.sign(np.diag(R))
    d[d == 0] = 1.0
    Q = Q * d
    return Q

def make_matrix_with_condition(n, kappa=10.0, structure="general", spectrum="geom", seed=None):
    """
    Crea una matriz A de tamaño n×n con número de condición ~kappa en norma 2.
    structure: "general" (no simétrica) o "spd" (simétrica definida positiva).
    spectrum:  "geom" reparte singular/propios en progresión geométrica entre [1/kappa, 1].
    """
    if kappa < 1:
        raise ValueError("kappa debe ser >= 1")
    rng = np.random.default_rng(seed)

    # Valores singulares/propios: de 1/kappa (mín) a 1 (máx)
    if spectrum == "geom":
        vals = np.geomspace(1.0, 1.0/kappa, num=n)  # descendente; max=1, min=1/kappa
        vals = np.sort(vals)[::-1]                  # garantizar orden descendente
    else:
        raise ValueError("spectrum no soportado (usa 'geom')")

    if structure == "general":
        U = _random_orthogonal(n, rng)
        V = _random_orthogonal(n, rng)
        A = U @ np.diag(vals) @ V.T
    elif structure == "spd":
        Q = _random_orthogonal(n, rng)
        A = Q @ np.diag(vals) @ Q.T   # simétrica y definida positiva
    else:
        raise ValueError("structure debe ser 'general' o 'spd'")

    # Reescala opcional: no cambia kappa; útil si quieres ||A|| ~ 1, etc.
    # A = A / np.linalg.norm(A, 2)

    # Verificación empírica (puede diferir ~1e-12 por redondeo)
    s = np.linalg.svd(A, compute_uv=False)
    kappa_emp = s[0] / s[-1]
    return A, kappa_emp

def generate_system(N, kappa=10.0, structure="general", seed=None, x_known=None):
    """
    Genera (A, x_true, b) con cond_2(A) ~ kappa.
    - N: tamaño
    - structure: 'general' o 'spd'
    - x_known: si lo pasas, se usa; si no, se crea aleatorio.
    """
    rng = np.random.default_rng(seed)
    A, kappa_emp = make_matrix_with_condition(N, kappa=kappa, structure=structure, seed=seed)
    if x_known is None:
        x_true = rng.standard_normal(N)
    else:
        x_true = np.asarray(x_known, dtype=float)
        if x_true.shape != (N,):
            raise ValueError("x_known debe tener shape (N,)")
    b = A @ x_true
    return A, x_true, b, kappa_emp


In [5]:
N = 5
kappa_deseada = 100
A, x_true, b, kappa_emp = generate_system(N, kappa=kappa_deseada, structure="general", seed=42)

print("condición objetivo:", kappa_deseada)
print("condición empírica:", kappa_emp)

x_rec = np.linalg.lstsq(A, b, rcond=None)[0]
print("error relativo en x:", np.linalg.norm(x_rec - x_true) / np.linalg.norm(x_true))
print("A =\n", A)
print("x_true =\n", x_true)
print("b =\n", b)

condición objetivo: 100
condición empírica: 100.00000000000024
error relativo en x: 4.007648654119551e-15
A =
 [[-0.13081689  0.16657923  0.07723457 -0.03782061 -0.15396884]
 [ 0.11084154 -0.58528037 -0.33288669 -0.20974112 -0.0265564 ]
 [ 0.00141675  0.39448968  0.17665277  0.20730293  0.17369555]
 [ 0.08366047 -0.41891365 -0.20390799 -0.0688255   0.00174923]
 [-0.02311784 -0.02242665 -0.08526961 -0.13976344 -0.09035459]]
x_true =
 [ 0.30471708 -1.03998411  0.7504512   0.94056472 -1.95103519]
b =
 [ 0.10968479  0.24717976 -0.42116632  0.23998568 -0.00288329]


## **2.3 QR Factorization**

In [6]:
def get_Q_Matrix(householder_vectors):
    """
    Calculates Q matrix of QR factorization using Householder reflections.

    Parameters
    ----------
    householder_vectors : ndarray (n,n), Householder reflection vectors.
    each i-th column represents a Householder vector for the i-th step of the QR factorization.

    Returns
    -------
    Q : ndarray (n,n), Orthogonal matrix.
    """
    n = householder_vectors.shape[0]
    Q = np.eye(n)
    for i in range(n):
        u = householder_vectors[:n-i, i]
        H_tilde = get_householder_matrix(u)
        H_full = np.eye(n)
        H_full[i:, i:] = H_tilde
        Q = Q @ H_full
    return Q

def QR_Householder(A):
    """
    QR factorization using Householder reflections.

    Parameters
    ----------
    A : ndarray (n,n), input matrix.
    Returns
    -------
    Q,R : ndarray (n,n), Orthogonal matrix and upper triangular matrix.
    """
    if (A.shape[1] != A.shape[0]):
        print("Matrix is not square")
        return None, None
    
    n = A.shape[0]
    # We need to work with submatrices of A
    # and we don't want to modify the original Matrix
    A = A.copy()

    # Each Householder vector is 1 dimension smaller than the previous one,
    # So we use just the first n-i rows on each iteration 
    householder_vectors = np.zeros((n, n))

    for i in range(n):
        sub_A = A[i:, i:]
        # Get Column i of A
        col_i_of_A = sub_A[:,0]
        col_i_of_A = col_i_of_A.reshape(-1,1)

        # Compute the Householder vector
        alpha = -np.sign(col_i_of_A[0,0]) * np.linalg.norm(col_i_of_A)
        e1 = np.eye(col_i_of_A.shape[0])[:,0].reshape(-1,1)
        u = col_i_of_A - alpha * e1
        u = u / np.linalg.norm(u)
        # print(f"u en la iteracion {i}, {u.shape} =\n", u)

        # Save the Householder vector
        householder_vectors[0:n-i,i] = u.ravel()
        # print(f"holder_vectors en la iteracion {i} =\n", holder_vectors)
        
        # FIX RESPECTO A LA VERSIÓN DE LA CALSE ANTERIOR.
        A[i:, i:] -= 2 * np.outer(u, u.T @ A[i:, i:])

    Q = get_Q_Matrix(householder_vectors)
    R = np.triu(A)
    # R = A
    return Q,R

Q,R = QR_Householder(A)
# print("uni?:\n", Q.T@Q)
compare_numpy = True
if (N <= 5):
    # print("A =\n", A)
    print("R:\n", R)
    print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A - Q @ R))
    if compare_numpy:
        A_np, Q_np, R_np = A.copy(), Q.copy(), R.copy()
        print("Comparando con numpy...")
        Q_np, R_np = np.linalg.qr(A_np)
        print("R (numpy):\n", R_np)
        print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A_np - Q_np @ R_np))
else:
    print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A - Q @ R))
    if compare_numpy:
        A_np, Q_np, R_np = A.copy(), Q.copy(), R.copy()
        print("Comparando con numpy...")
        Q_np, R_np = np.linalg.qr(A_np)
        print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A_np - Q_np @ R_np))

R:
 [[ 0.1921833  -0.62770239 -0.32176976 -0.10684436  0.10239912]
 [ 0.          0.55489204  0.29328574  0.29399594  0.22344126]
 [ 0.          0.          0.08533463  0.10391032  0.02808917]
 [ 0.          0.          0.         -0.06340605 -0.03532516]
 [ 0.          0.          0.          0.          0.01733085]]
Reconstrucción ||A - Q R|| = 1.6793714193975857e-16
Comparando con numpy...
R (numpy):
 [[ 0.1921833  -0.62770239 -0.32176976 -0.10684436  0.10239912]
 [ 0.          0.55489204  0.29328574  0.29399594  0.22344126]
 [ 0.          0.          0.08533463  0.10391032  0.02808917]
 [ 0.          0.          0.         -0.06340605 -0.03532516]
 [ 0.          0.          0.          0.         -0.01733085]]
Reconstrucción ||A - Q R|| = 2.5547594848822963e-16


## **2.4 Arnoldi Householder**

Partimos de un sistema:
$$
A x = b
$$

Donde podemos definir el primer vector ortonormalizado:

$$
q_1 = \frac{b}{\| b \|}
$$

Y en base a ese sistema se puede obtener el sub-espacio de Krylov de la matriz A:
$$
\begin{align*}
\mathcal{K}(A,b) = \{b, Ab, A^2 b, ...\}
\end{align*}
$$

La cual tiene el mismo span que las bases ortonormalizadas:
$$
\begin{align*}
\mathcal{K}(A,b) &= \text{span}\{b, Ab, A^2 b, ...\} \\
                 &= \text{span}\{q_1, q_2, q_3, ...\} \\
                 &= \text{span}\{q_1, Aq_1, Aq_2, ...\}

\end{align*}
$$


El obejtivo de la iteración de Arnoldi es obtener una base ortonormalizada, a partir la base de Krylov:
$$
Q = \{q_1, q_2, q_3,...\}
$$

Donde se tiene la relación fundamental:
$$
A Q_m = Q_{m+1} H_m
$$

Donde $H$ es la matriz de Hessenberg de dimensiones $(m+1) \times m$


In [7]:
def householder_to_e1(vec):
    """
    return the u vector such that:
    Hu(vec) = ||vec||*e1
    where Hu is the Householder reflection matrix defined by u.

    Parameters
    ----------
    vec : ndarray (n,1), input vector.
    Returns
    -------
    u : ndarray (n,1), Householder vector.
    """
    n = vec.shape[0]
    v = vec.copy().reshape(-1,1)

    alpha = -np.sign(v[0]) * np.linalg.norm(v)
    e1 = np.zeros((n,1))
    e1[0] = 1.0
    u = v - alpha * e1
    u = u / np.linalg.norm(u)
    return u 

In [8]:
random_vec = np.random.randn(5)
print("random_vec =", random_vec)

u1 = householder_to_e1(random_vec)
print("u1 =", u1)

H1 = get_householder_matrix(u1)
print("H1 =\n", H1)

a1_transformed = H1 @ random_vec
print("H1 @ random_vec:\n",a1_transformed)


random_vec = [-0.04204057 -0.82765984 -2.13506977 -2.34911744  1.08476944]
u1 = [[-0.71139522]
 [-0.16834558]
 [-0.43427207]
 [-0.47780926]
 [ 0.22064154]]
H1 =
 [[-0.01216632 -0.23952048 -0.61787815 -0.67982244  0.31392667]
 [-0.23952048  0.94331953 -0.14621557 -0.16087415  0.07428805]
 [-0.61787815 -0.14621557  0.62281553 -0.41499843  0.19163691]
 [-0.67982244 -0.16087415 -0.41499843  0.54339663  0.21084914]
 [ 0.31392667  0.07428805  0.19163691  0.21084914  0.90263463]]
H1 @ random_vec:
 [ 3.45548675e+00 -9.60243090e-17 -4.19596175e-16 -2.93877024e-16
  2.13267848e-16]


In [9]:
def householder_arnoldi(A, r0, m, verbose=False):
    # Inicializacion
    n = A.shape[0]
    V = np.zeros((n, m + 1))
    H_bar = np.zeros((m + 1, m))
    
    # Lista para almacenar los vectores de Householder
    w_vectors = []

    # Z son los vectores a transformar para formar la matriz H_bar
    z = r0.copy()

    # El bucle va de j=1 a m+1
    for j in range(1, m + 2):
        if verbose:
            print(f"\n =============== Iteración j={j} =============== ")
    # PASO 1: Cálculo del vector de Householder w_j
    # tal que mande z_j a un multiplo de e_j
    # (sus primera j-1 componentes son cero)
    sub_z = z[j - 1:]
    alpha = np.linalg.norm(sub_z)

    if alpha == 0:
        v = np.zeros_like(sub_z)
    else:
        e1_sub = np.zeros_like(sub_z)
        e1_sub[0] = 1.0
        v = sub_z + np.copysign(alpha, sub_z[0]) * e1_sub
        v /= np.linalg.norm(v)
    # Vector w_j completo (con ceros arriba)
    w_j = np.zeros(n)
    w_j[j - 1:] = v
    w_vectors.append(w_j)
    if verbose:
        print("PASO 1: Calcular vector de Householder w_j")
        print(f"Vector w_{j} = \n{w_j} \n")

    # PASO 2: Aplicación de transformaciones a z
    h_full = z - 2 * w_j * (w_j.T @ z)
    # La columna j-2 de H_bar se calcula en la iteración j.
    if j > 1 and j <= m + 1:
        H_bar[:, j - 2] = h_full[:m + 1]
    if verbose:
        print("PASO 2: Aplicar transformaciones a z")
        print("Vector original z = \n", z, "\n")
        print("Vector transformado h_full = \n", h_full, "\n")
        print(f"H_bar actualizado (columna {j - 2}):\n", H_bar, "\n")
        if j == 1:
            print("No se agrega el vector porque es la transformación del primer vector de R y a nosotros nos interesa H_bar\n")

    # PASO 3: Reconstrucción de v_j aplicando las reflexiones en orden inverso
    v_j = np.zeros(n)
    v_j[j - 1] = 1.0
    for i in range(j, 0, -1):
        w_i = w_vectors[i - 1]
        v_j = v_j - 2 * w_i * (w_i.T @ v_j)
        V[:, j - 1] = v_j
    if verbose:
        print("PASO 3: Reconstruir v_j aplicando reflexiones inversas")
        print(f"Nuevo vector v_{j} = \n{v_j}")
        print(f"Agregando v_{j} a V[:, {j - 1}] \n{V}")


    # PASO 4: Preparación de z para la siguiente iteración
    if j <= m:
        Av = A @ v_j
        z_next = Av.copy()
        for i in range(1, j + 1):
            w_i = w_vectors[i - 1]
            z_next = z_next - 2 * w_i * (w_i.T @ z_next)
            z = z_next
        if verbose:
            print("PASO 4: Preparar z para la siguiente iteración")
            print(f"z_{j+1} = A v_{j} reflejado = \n{z}\n")
    V_m = V[:, :m]
    V_m_plus_1 = V
    return V_m, V_m_plus_1, H_bar



np.random.seed(0)
n = 3
A = np.random.rand(n, n)
r0 = np.random.rand(n)
r0 = r0 / np.linalg.norm(r0) # Normalizar el vector inicial
m = 2 # Número de iteraciones

# Ejecutar el algoritmo
V_m, V_m_plus_1, H_bar = householder_arnoldi(A, r0, m, verbose=True)
# Verificar la propiedad fundamental de Arnoldi: A * Vm = V_{m+1} * H_bar
LHS = A @ V_m
RHS = V_m_plus_1 @ H_bar
print("Matriz V_m (base ortonormal):")
print(np.round(V_m, 4))
print("\nMatriz V_{m+1} (base ortonormal extendida):")
print(np.round(V_m_plus_1, 4))
print("\nMatriz de Hessenberg H_bar:")
print(np.round(H_bar, 4))
# Comprobar la ortonormalidad de V_m (debería ser cercano a la identidad)
print("\nVerificación de ortonormalidad (V_m^T * V_m):")
print(np.round(V_m.T @ V_m, 4))


print("\nVerificación de ortonormalidad (V_{m+1}^T * V_{m+1}):")
print(np.round(V_m_plus_1.T @ V_m_plus_1, 4))

# Comprobar el error
error = np.linalg.norm(LHS - RHS)
print(f"\nError ||A*V_m - V_{{m+1}}*H_bar||: {error:.2e}")


 =============== Iteración j=1 =============== 

 =============== Iteración j=2 =============== 

 =============== Iteración j=3 =============== 
PASO 1: Calcular vector de Householder w_j
Vector w_3 = 
[0. 0. 1.] 

PASO 2: Aplicar transformaciones a z
Vector original z = 
 [0.37356329 0.77132859 0.51526951] 

Vector transformado h_full = 
 [ 0.37356329  0.77132859 -0.51526951] 

H_bar actualizado (columna 1):
 [[ 0.          0.37356329]
 [ 0.          0.77132859]
 [ 0.         -0.51526951]] 



IndexError: list index out of range